# Generate spectrograms for all downloaded tracks

Find all tracks, then use librosa to generate mel spectrograms for all tracks.

In [10]:
import glob
from multiprocessing import Pool
from tqdm import tnrange, tqdm_notebook
import os
from tempfile import mktemp
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import subprocess
import ffmpeg
import librosa
import librosa.display
import numpy as np
from utils import make_logger
import logging

logger = make_logger('generatespec', 'log/generatespec.log')

dpi = 92
resolution = 64

def make_spectrogram(filename, replace=False):
    outname = os.path.abspath(filename.replace('.mp3', '.png').replace('.m4a', '.png'))
    if os.path.isfile(outname) and not replace:
        return
        
    samples, sample_rate = librosa.load(filename)
    
    S = librosa.feature.melspectrogram(y=samples, sr=sample_rate)

    fig, ax = plt.subplots(frameon=False)
    fig.set_size_inches(resolution * 4 / dpi, resolution / dpi)
    
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000, x_axis='time', cmap='gray_r',)
    fig.savefig(outname, dpi=92)
    plt.close()


files = glob.glob(os.getcwd() + '/tracks/**/*.mp3', recursive=True) \
    + glob.glob(os.getcwd() + '/tracks/**/*.m4a', recursive=True)

logger.info("Generating spectrogram data for all audio samples...")
p = Pool(10)
r = list(tqdm_notebook(p.imap(make_spectrogram, files), total=len(files)))
logger.info("Generated spectrogram data for audio samples successfully")

Generating spectrogram data for all audio samples...


Generated spectrogram data for audio samples successfully


KeyboardInterrupt: 